# Robot Tracking

## It is recommended that you finish the exercises from Monday, before starting the project.

For this project you are given a video of some mobile robots (Robots.mp4). The task is now to track only the robots that are moving. Try to use both sparse and dense optical flow and compare the results.

For sparse optical flow, draw the tracked keypoints onto each frame and try to show the frames fast enough, such that it looks like a video.

For dense optical flow, represent the movement in any way you see fitting. For example by making a new image with the colors of each pixel representing the movement.


## Handling a video with OpenCV

With OpenCV we can load a video using the cv2.VideoCapture function:

```python
cap = cv2.VideoCapture('Robots.mp4')                       
````

When we need a single frame of the video, we can then use the read function of the created cv2.VideoCapture object:

```python
ret, frame = cap.read()
```

The variable ret will be either True or False indicating if the frame was read correctly. The variable frame will then be the first frame of the video. If we call the same function again, it will return the second frame of the video, and so on.

Thus, if we keep calling the read function in a loop, we will be able to access every frame in the video.

At the end of your program you should add a line that releases the video capture:

```python
cap.release()
```
  

So far, when we have displayed images in JupyterLab, we have used Matplotlib (plt.imshow), instead of the standard OpenCV way cv2.imshow (OpenCV: High-level GUI). The reason for this is that Matplotlib will show the image right below your code, while OpenCV will open a new window with the image. In this project however, it might be beneficial to use cv2.imshow as we would like to display many frames after each other. So, when displaying images, try doing it the following way:

```python
cv2.imshow('image', image)
cv2.waitKey(20)
```

Notice we have to give the window a name when using cv2.imshow. The second line pauses the script for 20 milliseconds before moving on.

When using cv2.imshow in JupyterLab it is important to remove the image window in the end, otherwise Jupyter might crash. This is done by adding the following line to the very end of your code:

```python
cv2.destroyAllWindows()
```
 
Challenge (optional)

If you have finished the other tasks and still have some time left, try the following:

 - Pick one of the moving robots and track only that one.

 - Try the same for the other video (Challenge.mp4).

You can for example try to detect which robot is which by using a feature descriptor and matching method (SIFT, ORB, etc.).

import cv2
import numpy as np
from matplotlib import pyplot as plt


In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
cap = cv2.VideoCapture('Robots.mp4')

while True:
    ret, frame = cap.read()

    if not ret:
        break

    cv2.imshow('Original Video', frame)
    cv2.waitKey(20)

cap.release()
cv2.destroyAllWindows()


In [ ]:
cap = cv2.VideoCapture('Robots.mp4')

ret, frame = cap.read()
gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
feat1 = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.3, minDistance=7)

# Store the original feature points
original_points = feat1.copy()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    gray2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(gray1, gray2, feat1, None)

    # Select good points
    good_new = feat2[status == 1]
    good_original = original_points[status == 1]

    # Create a copy of the frame for drawing
    frame_copy = frame.copy()

    # Draw the tracks from original positions to current positions
    for i, (new, original) in enumerate(zip(good_new, good_original)):
        a, b = new.ravel().astype(int)
        c, d = original.ravel().astype(int)
        cv2.line(frame_copy, (a, b), (c, d), (0, 255, 0), 2)
        cv2.circle(frame_copy, (a, b), 5, (0, 255, 0), -1)

    cv2.imshow('Sparse Optical Flow', frame_copy)
    cv2.waitKey(20)

    # Update the previous frame and points for next iteration
    gray1 = gray2.copy()
    feat1 = good_new.reshape(-1, 1, 2)

    # Keep original points aligned with current tracking
    original_points = good_original.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()
